# Exploring the SQuAD dataset

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [7]:
explore_df = dataset["train"].to_pandas()

In [14]:
text = explore_df.head(1)["context"][0]
text

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

## Summarization

In [13]:
from transformers import pipeline

pipe = pipeline("summarization", model="sshleifer/distilbart-xsum-12-6")

In [16]:
pipe(text, min_length=60)

[{'summary_text': ' Beyoncé is one of the world\'s most successful and influential singers, with her hits including "Crazy in Love" and "Baby Boy", and her first solo album, Dangerously in Love, which was released in July 2012, and is due to be released in June 2017...'}]

Will summarization be optimal, given that we want to test users on **detail**, not just general points?